In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pulp import *
import pandas as pd
from sklearn.neighbors import DistanceMetric
from math import radians
import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx

In [2]:
import osmnx as ox
ox.config(use_cache=True, log_console=True)

In [3]:
# Number of Candidate DCs
DC_N = 4

# List Creator for DCs
def createList_DC(r1, r2):
    return list(['DC%d'%x for x in range(r1, r2+1)])

#create an index
DC_ID = createList_DC(1, DC_N)

#Gi = fixed cost for establishing depot i
fixed_cost_DC = [1500] * DC_N 

#Maximum Throughput at depot i = Vi
capacity_DC = [100] * DC_N

#Variable Warehousing Cost (Picking)
varCost_DC = [12, 15, 20, 30]

#Position of the DCs
lat_DC = [49.794783, 49.81613668, 49.7881377, 49.8226541]
lon_DC = [9.906499, 9.897654496, 9.9314981, 9.9245535]

dc_tuples = list(zip(DC_ID, fixed_cost_DC, capacity_DC, varCost_DC, lat_DC, lon_DC))

set_of_all_DC = pd.DataFrame(dc_tuples, columns = ["DC_ID", "fixed_cost_DC", "capacity_DC", "varCost_DC", "lat", "lon"])
set_of_all_DC.set_index("DC_ID", inplace = True)

# Define I
I = set_of_all_DC.index.values
I

set_of_all_DC

,fixed_cost_DC,capacity_DC,varCost_DC,lat,lon
DC_ID,,,,,
DC1,1500,100,12,49.794783,9.906499
DC2,1500,100,15,49.816137,9.897654
DC3,1500,100,20,49.788138,9.931498
DC4,1500,100,30,49.822654,9.924554


In [4]:
# Sample Customers from osmnx

# Define Customer N and Demand per
customers = 25
demand_per_customer = 5

# Function to create a Customer Index List

def createList_C(r1, r2):
    return list(['C%d'%x for x in range(r1, r2+1)])

# Get all nodes in Würzburg from OSMNX which will be used to sample a certain amount of customers
G = ox.graph_from_place("Wuerzburg, Germany", network_type = "drive")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

# Create nodes_df which is the basis for our customer df
nodes_df = gdf_nodes[["y", "x"]].copy()
nodes_df.columns = ["lat", "lon"]

#Sample from nodes_df
sample_nodes_df = nodes_df.sample(n = customers, random_state= 4)
Nodes_S = sample_nodes_df.index.values
# Create Nodes_S which will help in plotting the solutions within osmnx
Nodes_S.tolist()

#DF Manipulation
C_ID = createList_C(1,customers)
osmid = list(sample_nodes_df.index.values)
sample_nodes_df = sample_nodes_df.rename(index=dict(zip(osmid,C_ID)))
sample_nodes_df.index.name = "C_ID"

#Create final demand column for customer DF
mylist = [demand_per_customer] * customers

set_of_all_customers = sample_nodes_df.copy()
set_of_all_customers['Demand_C'] = mylist
J = set_of_all_customers.index.values
set_of_all_customers.head()

,lat,lon,Demand_C
C_ID,,,
C1,49.800625,9.914932,5
C2,49.804163,9.918820,5
C3,49.800478,9.933612,5
C4,49.786208,9.944269,5
C5,49.792946,9.930499,5


In [5]:
# Construct set of all vehicles K

#Vehicle Count
V_N = 5

# Vehicle List Creator
def createList_V(r1, r2):
    return list(['V_%d'%x for x in range(r1, r2+1)])

# create index
V_ID = createList_V(1, V_N)

# Vehicle Capacity Qk
capacity_V = [100] * V_N

# fixed cost of using Vehicle Fk
fixed_cost_V = [5] * V_N

v_tuples = list(zip(V_ID, capacity_V, fixed_cost_V))

set_of_all_vehicles = pd.DataFrame(v_tuples, columns = ["V_ID", "capacity_V", "fixed_cost_V"])
set_of_all_vehicles.set_index("V_ID", inplace=True)
K = set_of_all_vehicles.index.values
set_of_all_vehicles

,capacity_V,fixed_cost_V
V_ID,,
V_1,100,5
V_2,100,5
V_3,100,5
V_4,100,5
V_5,100,5


In [6]:
# Create a distance matrix with all DCs and Customers

# Concat the DC and Customer DF
DF = pd.concat([set_of_all_DC, set_of_all_customers], axis = 0)
DF = DF.drop(DF.columns[[0,1,2,5]], axis = 1)
DF['C_ID'] = DF.index

#Convert lat and lon in to radians
DF['lat'] = np.radians(DF['lat'])
DF['lon'] = np.radians(DF['lon'])

# Define dist parameter as haversine
dist = DistanceMetric.get_metric("haversine")
# Convert to numpy array
DF[["lat", "lon"]].to_numpy()

#Create the distance matrix py pairwise comparison
dist_matrix = pd.DataFrame(dist.pairwise(DF[["lat", "lon"]].to_numpy())*6378100, columns=DF.C_ID.unique(), index=DF.C_ID.unique())
dist_matrix = dist_matrix.round(1)
dist_matrix

,DC1,DC2,DC3,DC4,C1,C2,C3,C4,C5,C6,...,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25
DC1,0.0,2460.5,1942.9,3362.8,888.8,1369.0,2048.7,2877.3,1736.7,2812.8,...,1261.3,6212.5,4185.6,1835.4,385.3,4853.9,3130.9,2784.3,2386.1,7442.3
DC2,2460.5,0.0,3953.1,2063.7,2126.6,2022.0,3116.3,4724.1,3497.5,5076.6,...,1810.3,6894.7,4925.6,3868.1,2433.4,5421.0,5222.9,5200.1,4616.7,9852.8
DC3,1942.9,3953.1,0.0,3874.6,1830.1,2003.1,1382.0,942.6,540.1,1223.2,...,2165.3,4677.5,2835.4,108.7,2288.1,3528.1,1271.0,1720.8,782.4,6065.7
DC4,3362.8,2063.7,3874.6,0.0,2547.8,2099.2,2553.0,4297.3,3334.5,5095.5,...,2119.2,5255.9,3519.9,3836.2,3566.2,3832.2,4984.0,5515.4,4655.2,9910.0
C1,888.8,2126.6,1830.1,2547.8,0.0,482.9,1342.3,2649.5,1407.8,2958.6,...,439.7,5533.7,3484.1,1742.7,1207.8,4122.4,3101.1,3161.3,2496.7,7773.3
C2,1369.0,2022.0,2003.1,2099.2,482.9,0.0,1139.2,2709.1,1504.4,3197.9,...,252.6,5241.4,3193.9,1933.4,1670.5,3800.4,3251.4,3493.0,2736.5,8043.7
C3,2048.7,3116.3,1382.0,2553.0,1342.3,1139.2,0.0,1763.4,867.7,2574.4,...,1388.1,4197.2,2152.0,1371.7,2427.5,2808.6,2431.4,3096.6,2157.8,7358.7
C4,2877.3,4724.1,942.6,4297.3,2649.5,2709.1,1763.4,0.0,1241.8,1194.7,...,2913.9,3946.8,2337.3,1051.3,3228.6,2986.9,750.7,1969.0,1043.3,5657.7
C5,1736.7,3497.5,540.1,3334.5,1407.8,1504.4,867.7,1241.8,0.0,1762.5,...,1690.0,4571.6,2618.5,510.1,2113.7,3312.6,1747.9,2232.7,1321.5,6599.8
C6,2812.8,5076.6,1223.2,5095.5,2958.6,3197.9,2574.4,1194.7,1762.5,0.0,...,3336.1,5052.5,3531.8,1273.6,3074.7,4175.9,637.6,801.5,462.9,4848.9


<font size="6">**Define the Model**</font><br>
![Wu_New](https://i.imgur.com/DeVQzoi.png)<br>

![wu_model2](https://i.imgur.com/h6Dng6i.png)

In [7]:
model = LpProblem("LRP", LpMinimize)

<font size = 5>**Declare Decision Variables**</font>

In [8]:
# Define Xijk Decision Variable = Connections between Customers/DCs in specific vehicles
x = LpVariable.dicts(name = "x", indexs=([*I,*J],[*I,*J],K), cat = "Binary")

# Define yi Decision Variable = Depot Open/Closed
y = LpVariable.dicts(name = "y", indexs=I, lowBound = 0, upBound = 1, cat = "Binary")

# Define zij Decision Variable = Customer allocated to DC or not
z = LpVariable.dicts(name = "z", indexs = (I,J), cat = "Binary")

# Define Axuliary Variable for sub-tour elimination
U = [(l,k) for l in J for k in K]
u = LpVariable.dicts(name = "u", indexs = U, lowBound = 0, cat = "Continuous")

In [9]:
# Objective Function
# Gi * yi
fixedCost_depot = lpSum([y[i] * set_of_all_DC.loc[i].fixed_cost_DC for i in I])

# Cij * Xijk

variableCosts_transp = lpSum([x[i][j][k] * dist_matrix.loc[i,j] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])

#VCi * dj*zj 
variableCosts_DC = lpSum(set_of_all_DC.loc[i].varCost_DC * z[i][j] * set_of_all_customers.loc[j].Demand_C for j in J for i in I)

# Fk * Xijk
fixedCost_vehicle = lpSum([[x[i][j][k]] * set_of_all_vehicles.loc[k].fixed_cost_V  for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])

# Add to model
model += fixedCost_depot + variableCosts_transp + variableCosts_DC + fixedCost_vehicle

In [10]:
# Constraints

# (2)
for j in J:
    model += lpSum([[x[i][j][k]] for i in [*I,*J] for k in K if i!=j]) == 1
    
# (3)
for k in K:
    model += lpSum([set_of_all_customers.loc[j].Demand_C * x[i][j][k] for i in [*I,*J] for j in J if i!=j]) <= set_of_all_vehicles.loc[k].capacity_V
# (4)
for l in J:
    for j in J:
        if l!=j:
            for k in K:
                model += (u[l,k] - u[j,k] + (len(set_of_all_customers) * [x[l][j][k]])) <= len(set_of_all_customers) - 1
# (5)
for i in [*I,*J]:
    for k in K:
        model+= lpSum([[x[i][j][k]] for j in [*I,*J] if i!=j]) - lpSum([[x[j][i][k]] for j in [*I,*J] if i!=j]) == 0
# (6)
for k in K:
    model += lpSum([[x[i][j][k]] for i in I for j in J]) <= 1
# (7)
for i in I:
    model += lpSum(z[i][j] * set_of_all_customers.loc[j].Demand_C for j in J) - (set_of_all_DC.loc[i].capacity_DC * y[i]) <= 0
# (8)
for i in I:
    for j in J:
        for k in K:
            model += lpSum([x[i][u][k] + x[u][j][k] for u in [*I,*J]]) - z[i][j] <= 1

In [ ]:
model.solve(GUROBI_CMD(gapRel = 0.1))
LpStatus[model.status]

In [ ]:
#Problem Config Log
# Instance 1: Stuck at 29.2% Gap after 67.000 seconds
# Customers:50
# Customer Demand: 5 
# Potential DCs = 5
# DC Capacity = 100
# Fixed Cost DC = 1500$
# Variable Cost DC = [12, 15, 20, 30, 40]$
# Vehicles: 4
# Vehicle Capacity: 100
# Fixed Cost Vehicle per Route: 5$

# Instance 2: Solved with 10% Gap after 
# Customers: 25
# Customer Demand: 5 
# Potential DCs = 4
# DC Capacity = 100
# Fixed Cost DC = 1500$
# Variable Cost DC = [12, 15, 20, 30, 40]$
# Vehicles: 5
# Vehicle Capacity: 100
# Fixed Cost Vehicle per Route: 5$


In [ ]:
# Check if binary variables are alright
for variable in model.variables():
    if variable.varValue >= 0.1:
        print("{} = {}".format(variable.name, variable.varValue))

In [ ]:
# DC Printer
print("The following DCs are established:")
for i in I:
    if y[i].varValue >= 0.1:
        print("-{}".format(set_of_all_DC.loc[i].name)) 

In [ ]:
# new cost printer
fixedC = sum([y[i].varValue * set_of_all_DC.loc[i].fixed_cost_DC for i in I])
varC = sum([x[i][j][k].varValue *  dist_matrix.loc[i,j] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])
fixedCV = sum([x[i][j][k].varValue * set_of_all_vehicles.loc[k].fixed_cost_V  for i,j,k in R])
varDC = sum(set_of_all_DC.loc[i].varCost_DC * z[i][j].varValue * set_of_all_customers.loc[j].Demand_C for j in J for i in I)
print("Distance traveled: {}m \nFixed Costs for Depots: {}€ \nFixed Cost for Vehicles: {}€ \n Variable Warehousing Cost {}€".format(varC, fixedC, fixedCV, varDC))

In [ ]:
# new route count printer
Route_v1 = []
Route_v2 = []
v1 = sum([x[i][j][k].varValue for i in [*I,*J] for j in [*I,*J] for k in K if k == ("V_1")])
v2 = sum([x[i][j][k].varValue for i in [*I,*J] for j in [*I,*J] for k in K if k == ("V_2")])
Route_v1.append(v1)
Route_v2.append(v2)
print("Vehicle 1 has to drive {} routes".format(Route_v1))
print("Vehicle 2 has to drive {} routes".format(Route_v2))

In [ ]:
# new route printer (not using the R_df for distances anymore and not using R as index)
print("The following Routes have to be driven to satisfy all demand:")
for i in [*I,*J]:
    for j in [*I,*J]:
        for k in K:
            if x[i][j][k].varValue >= 0.1:
                print("From {} to {} in Vehicle {} for length {}m".format(i,j,k, dist_matrix.loc[i,j])) 

In [ ]:
# Route Order Printer - still WIP since i and k are hardcoded here. Would be nice to do this without having to call specific Warehouses/Vehicles since the Model will grow in size substantially
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and i == ("DC_1") and k == ("V_2"):
        print("Vehicle {} starts from {} and drives to {} with route length {}m".format(k,i,j, dist_matrix.loc[i,j]))
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and k == ("V_2") and i != ("DC_1"):
        print("Then drives from {} to {} with route length {}m".format(i,j, dist_matrix.loc[i,j]))

In [ ]:
# Route Order Printer minimalistic Vehicle 2
print("Vehicle 2 drives the following route to satisfy all demand:")
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and i == ("DC_1") and k == ("V_2"):
        print("{} -> {}: route length {}m".format(i,j, dist_matrix.loc[i,j]))
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and k == ("V_2") and i != ("DC_1"):
        print(" {} -> {}: route length {}m".format(i,j, dist_matrix.loc[i,j]))

In [ ]:
# First show the potential DCs and Customers
warnings.filterwarnings("ignore")

place = {"city": "Wuerzburg", "country": "Germany"}
G = ox.graph_from_place(place, network_type = "drive")
#C3 49.758535, 9.932180
# DC Coords
coord_DC1 = (49.794783, 9.906499)
coord_DC2 = 
coord_DC3 = 
coord_DC4 =
coord_DC5 =


DC_1 = ox.get_nearest_node(G, coord_DC1)
DC_2 = ox.get_nearest_node(G, coord_DC2)
DC_3 = ox.get_nearest_node(G, coord_DC3)
DC_4 = ox.get_nearest_node(G, coord_DC4)
DC_5 = ox.get_nearest_node(G, coord_DC5)


nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)


DCs = [DC_1, DC_2, DC_3, DC_4, DC_5]

nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

ns = []
for node in G.nodes():
    if node in DCs:
        ns.append(150)
    elif node in Nodes_S:
        ns.append(25)
    else:
        ns.append(0)
        
nc = []
for node in G.nodes():
    if node in DCs: 
        nc.append("red")
    elif node in Nodes_S:
        nc.append("yellow")
    else:
        nc.append("white")
        

nc = []
for node in G.nodes():
    if node == DC_1:
        nc.append("red")
    elif (node == C1 or node == C2 or node == C3 or node == C4 or node == C5):
        nc.append("yellow")
    else:
        nc.append("white")
        

fig, ax = ox.plot_graph(G, node_size = ns, edge_linewidth = 0.5, node_color = nc, figsize = (22,22), bgcolor = "black")